# Download and import ImageAI

In [ ]:
!pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl

from imageai.Detection import ObjectDetection
import os

# Download dataset

In [ ]:
!wget https://www.di.ens.fr/willow/teaching/recvis18/assignment3/bird_dataset.zip
!unzip -q bird_dataset.zip
!rm bird_dataset.zip

# Download and load RetinaNet

In [ ]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("resnet50_coco_best_v2.0.1.h5")
detector.loadModel()

# Extract birds

In [ ]:
!rm -rf bird_dataset_ex

In [ ]:
from tqdm import tqdm
import glob
from shutil import copyfile
from PIL import Image

custom_objects = detector.CustomObjects(bird=True)

In [ ]:
for path in tqdm(sorted(glob.glob("bird_dataset/**/*.jpg", recursive=True))):
  path_ex = path.replace("bird_dataset", "bird_dataset_ex")
  if os.path.isfile(path_ex):
    continue
  
  dirname = os.path.dirname(path_ex)
  if not os.path.isdir(dirname):
    os.makedirs(dirname)

  try:
    detections = detector.detectCustomObjectsFromImage(
        custom_objects=custom_objects,
        input_image=path,
        output_image_path=path_ex,
        minimum_percentage_probability=40)

    detections = list(sorted(detections, key=lambda d: d["percentage_probability"], reverse=True))
  except Exception:
    detections = []  
  
  if len(detections) > 0:
    pts = detections[0]["box_points"]
    
    α = 0.1
    
    w = pts[2] - pts[0]
    h = pts[3] - pts[1]
    pts[0] = max(0, pts[0] - α*w)
    pts[1] = max(0, pts[1] - α*h)
    pts[2] = pts[2] + α*w
    pts[3] = pts[3] + α*h
    
    Image.open(path).crop(pts).save(path_ex)
  else:
    print("\nNo bird in {}".format(path_ex))
    copyfile(path, path_ex)